In [2]:
import pandas as pd
import os
os.chdir(r"C:\Users\Gyu\Desktop\mywork\FastOnline\2. 탐색적 데이터 분석\데이터")

df = pd.read_csv("Instacart Market Basket Analysis.csv")
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [3]:
product_list_per_order = df.groupby('order_id')['product_id'].apply(list)

In [3]:
# product_list_per_order

order_id
1          [49302, 11109, 10246, 49683, 43633, 13176, 472...
36         [39612, 19660, 49235, 43086, 46620, 34497, 486...
38         [11913, 18159, 4461, 21616, 23622, 32433, 2884...
96         [20574, 30391, 40706, 25610, 27966, 24489, 39275]
98         [8859, 19731, 43654, 13176, 4357, 37664, 34065...
                                 ...                        
3421049           [40800, 17706, 33424, 17299, 26800, 34243]
3421056                    [5750, 9340, 21709, 16475, 12432]
3421058    [15629, 4347, 34466, 6244, 6858, 30316, 35578,...
3421063                         [49235, 13565, 14233, 35548]
3421070                                 [35951, 16953, 4724]
Name: product_id, Length: 131209, dtype: object

In [4]:
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order) 
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_)
one_hot_df.head() # 매우 희소 (sparse)

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
from mlxtend.frequent_patterns import *
frequent_item_df = apriori(one_hot_df, min_support = 0.05) # 1% 이상 구매한 상품만 대상으로 함
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by = 'confidence', ascending = False)

### 시퀀스 데이터에서의 연관 규칙

- 시퀀스 데이터에서 빈발 패턴은 순서가 고려되어야 함
- 윈도우 크기 내에 특정 이벤트가 발생했는지를 기준으로 패턴의 등장 여부 확인
- A -> B , B-> A가 다른 지지도를 가져 같은 항목 집합으로부터 규칙을 생성할 수 없음
    -> 따라서 개별 요소에 달느 요소를 추가하는 식으로 규칙을 직접 찾아야 함
    
패턴을 찾기 위해 DP활용
1. queue에 시퀀스를 넣음
2. 지지도/신뢰도 계산 후 일정 크기 이상인 조건을 만족하는 것을 다시 넣음

In [3]:
df = pd.read_csv("페이지내_사용자_이동.csv", engine = "python")
df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [4]:
# 주문별 카트에 추가한 순서를 고려하기 위해, 정렬이 필요
df.sort_values(by = ['고객ID', '순서'], inplace = True) 

In [5]:
page_set = df['방문 페이지'].unique()
page_set[:10]

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I'], dtype=object)

In [7]:
import numpy as np
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)
page_sequence_per_order.head()

고객ID
0    [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1    [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2    [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3    [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4    [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
Name: 방문 페이지, dtype: object

In [8]:
from itertools import product
def contain_pattern(record, pattern, L):
    output = False    
    if set(record) & set(pattern) != set(pattern): # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않으면
        return False    
    else:
        # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
        pattern_index_list = [np.where(record == item)[0] for item in pattern]
        
        ## 가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환
        # record = [A, B, C, A, C, C], pattern = [A, B], L = 1
        # A의 위치: [0, 3], B의 위치: [1]
        # 가능한 모든 조합: [0, 1], [3, 1] -> product로 산출
        # 가능한 모든 조합의 거리 차이: [1 - 0, 1 - 3] 중에 0번째 요소는 만족하므로 True
        
        for pattern_index in product(*pattern_index_list):
            distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
            if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1):
                output = True
                break
        
        return output

In [9]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.01, L = 1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가시킴
    for item in item_set:
        occurence = sequence_data.apply(contain_pattern, pattern = [item], L = L).sum()
        if occurence / len(sequence_data) >= min_support:
            queue.append([item])

    while queue:
        current_pattern = queue.pop() # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.apply(contain_pattern, pattern = current_pattern + [item], L = L).sum()
            if occurence / len(sequence_data) >= min_support: # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False 
                queue.append(current_pattern + [item])
        
        if check_maximum_frequent and len(current_pattern) > 1:
            maximum_frequent_sequence_item.append(current_pattern)        
    
    return maximum_frequent_sequence_item

In [10]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = {"부모":[], "자식":[], "지지도":[], "신뢰도":[]}
        
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산
        co_occurence = sequence_data.apply(contain_pattern, pattern = sequence_item, L = L).sum()
        support = co_occurence / len(sequence_data)
        if co_occurence > min_support:
            for i in range(len(sequence_item)-1, 0, -1): # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(contain_pattern, pattern = antecedent, L = L).sum()
                
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
            
    return pd.DataFrame(result)

In [11]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(page_set, page_sequence_per_order, min_support = 0.02, L = 1)

In [12]:
result = generate_association_rules(maximum_frequent_sequence_item, page_sequence_per_order, min_support = 0.01, min_confidence = 0, L = 1)
result.sort_values(by = ['지지도', '신뢰도'], ascending = False)

,부모,자식,지지도,신뢰도
160,[페이지B],[페이지J],0.07,0.155556
59,[페이지A],[페이지C],0.07,0.148936
109,[페이지J],[페이지D],0.07,0.142857
115,[페이지J],[페이지C],0.07,0.142857
75,[메인],[페이지H],0.06,0.150000
...,...,...,...,...
205,[페이지C],"[메인, 페이지A, 페이지J]",0.02,0.037037
207,[페이지C],"[메인, 페이지E]",0.02,0.037037
209,[페이지C],"[페이지J, 페이지I]",0.02,0.037037
214,[페이지C],"[페이지E, 메인]",0.02,0.037037


### 시계열 데이터에 대한 연관규칙 탐색

- 시계열은 연속형이어서 패턴을 찾으려면 이산화가 필요 -> SAX 사용

SAX
- 윈도우 분할, 윈도우별 대표값 계산, 알파벳 시퀀스로 변환

In [14]:
df = pd.read_excel("2018-20_코스피지수.xlsx", parse_dates = True) # 날짜 형식 그대로
df.head()

,날짜,종가
0,2020-07-17,2201.19
1,2020-07-16,2183.76
2,2020-07-15,2201.88
3,2020-07-14,2183.61
4,2020-07-13,2186.06


In [15]:
# 순서를 고려하기 위해, 정렬이 필요
df.sort_values(by = ['날짜'], inplace = True) 

In [16]:
def segmentation(x, w, a): # x: time series sample, w: window size, a: alphabet size
    window_mean = x[:w*int(len(x)/w)].reshape(w, -1).mean(axis = 1)
    if len(x) % w != 0:    
        window_mean = np.hstack([window_mean, x[w*int(len(x)/w):].mean()]) # 남겨진 부분은 그 부분만 평균해서 window_mean에 추가
    
    return window_mean

def find_break_points(wmv, a): # wmv:  window mean vector
    break_points = [np.quantile(wmv, (i+1)/a) for i in range(a)]
    return break_points

def conversion_window(wv, break_points): # wv: window mean vector 
    alphabet_sequence = []
    for e in wv:
        alphabet = 65
        for bp in break_points:
            if e < bp:
                alphabet_sequence.append(chr(alphabet))
                break
            else:
                alphabet += 1   
                
    return alphabet_sequence

In [17]:
KOSPI = df['종가'].values
window_mean_vector = segmentation(x = KOSPI, w = 100, a = 5)

In [18]:
window_mean_vector

array([2282.725     , 2284.32      , 2291.51      , 2293.9475    ,
       2273.465     , 2251.4475    , 2287.11      , 2310.595     ,
       2300.5325    , 2284.09      , 2304.1175    , 2331.6275    ,
       2316.5025    , 2219.9075    , 2154.9       , 2143.095     ,
       2046.02      , 2041.21      , 2084.465     , 2076.4525    ,
       2090.9       , 2071.75      , 2104.65      , 2104.07      ,
       2066.2725    , 2074.5325    , 2063.865     , 2026.8725    ,
       2016.58      , 2067.02      , 2108.655     , 2128.7975    ,
       2186.1475    , 2197.195     , 2199.6325    , 2210.5925    ,
       2229.58      , 2200.03      , 2154.2325    , 2159.345     ,
       2179.7725    , 2156.5575    , 2153.5575    , 2207.5025    ,
       2223.96      , 2237.7925    , 2213.585     , 2197.4575    ,
       2188.5175    , 2092.725     , 2067.995     , 2057.7525    ,
       2038.79      , 2061.4175    , 2098.095     , 2097.        ,
       2127.005     , 2127.1075    , 2114.1275    , 2071.3925 

In [19]:
bps = find_break_points(window_mean_vector, 5)
bps

[2064.98, 2090.9, 2154.2325, 2216.0199999999995, 2331.6275]

In [21]:
alphabet_sequence = conversion_window(window_mean_vector, bps)
alphabet_sequence[:10]

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']

In [25]:
# 문자열로 변환 (하나의 리스트만 대상으로 하기에, 이렇게 하는 것이 더 수월. 단, L = 1으로만 한정)
alphabet_sequence = '-'.join(alphabet_sequence)
alphabet_sequence

'E-E-E-E-E-E-E-E-E-E-E-E-E-D-C-A-A-B-B-C-B-C-C-B-B-A-A-A-B-C-C-D-D-D-D-E-D-D-D-D-D-C-D-E-E-D-D-D-C-B-A-A-A-C-C-C-C-C-B-B-B-B-A-A-A-A-A-A-A-A-B-B-B-A-A-B-B-B-C-C-C-C-C-B-B-C-D-D-D-D-E-E-D-C-E-E-D-B-A-A'

패턴 찾기

In [26]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.01, L = 1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가시킴
    for item in item_set:
        # 문자열이므로 단순히 count method로 구현 가능
        occurence = sequence_data.count(item)
        if occurence / (sequence_data.count('-') + 1) >= min_support:
            queue.append(item)

    while queue:
        current_pattern = queue.pop() # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.count(current_pattern + '-' + item)
            if occurence / (sequence_data.count('-') + 1) >= min_support: # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False 
                queue.append(current_pattern + '-' + item)
        
        if check_maximum_frequent:
            maximum_frequent_sequence_item.append(current_pattern)        
    
    return maximum_frequent_sequence_item

def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = {"부모":[], "자식":[], "지지도":[], "신뢰도":[]}
        
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산        
        co_occurence = sequence_data.count(sequence_item)
        support = co_occurence / (sequence_data.count('-') + 1)
        if co_occurence > min_support:
            for i in range(sequence_item.count('-') -1, 0, -1): # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = '-'.join(sequence_item.split('-')[:i])
                consequent = '-'.join(sequence_item.split('-')[i:])
                antecedent_occurence = sequence_data.count(antecedent)
               
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
            
    return pd.DataFrame(result)

In [27]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item([chr(65 + i) for i in range(5)], alphabet_sequence, min_support = 0.02, L = 1)

In [28]:
result = generate_association_rules(maximum_frequent_sequence_item, alphabet_sequence, min_support = 0.02, min_confidence = 0.1, L = 1)
result = result.sort_values(by = ['지지도', '신뢰도'], ascending = False)
result

,부모,자식,지지도,신뢰도
31,B-A,A-A,0.03,0.600000
22,C-C,B-B,0.03,0.500000
23,C,C-B-B,0.03,0.150000
24,C,B-B,0.03,0.150000
25,B,C-C,0.03,0.150000
26,B,B-C,0.03,0.150000
32,B,A-A-A,0.03,0.150000
10,C-D-D-D,D-E,0.02,1.000000
11,C-D-D,D-D-E,0.02,1.000000
13,C-C-C-C-C,B-B,0.02,1.000000
